In [2]:
from database.market import Market
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.adatabase import ADatabase

In [3]:
strat = ADatabase("cfa")

In [4]:
market = Market()
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()

In [ ]:
market.connect()
for industry in tqdm(sp500["GICS Sector"].unique()[5:]):
    industry_tickers = sp500[sp500["GICS Sector"]==industry]["Symbol"]
    industry_filings = market.retrieve_eps_filings(industry)
    for projection_value in tqdm([4]):
        start_year = int(2019 - 1)
        end_year = 2023 - 1
        for ticker in industry_tickers:
            try:
                ticker_prices = market.retrieve_ticker_prices("prices",ticker)
                ticker_prices = p.column_date_processing(ticker_prices)
                ticker_prices["year"] = [x.year for x in ticker_prices["date"]]
                ticker_prices["quarter"] = [x.quarter for x in ticker_prices["date"]]
                quarterlies = ticker_prices.groupby(["year","quarter"]).mean().reset_index()
                stuff = industry_filings.merge(quarterlies[["year","quarter","adjclose"]],on=["year","quarter"],how="left")
                rekt = stuff.copy()
                rekt["ticker"]=ticker
                rekt["y"] = rekt['adjclose'].shift(-projection_value)
                rekt.dropna(inplace=True)
                strat.connect()
                factors = [x for x in rekt.columns if x not in ["year","quarter","ticker","date","industry","y"]]
                for training_year in [7]:
                    for year in range(start_year,int(end_year)):
                        try:
                            training_set = rekt[(rekt["year"]<year) & (rekt["year"]>=year-training_year)].reset_index()
                            prediction_set = rekt[(rekt["year"]==year)]
                            refined = {"X":training_set[factors],"y":training_set["y"]}
                            models = m.regression(refined)
                            for row in models.iterrows():
                                try:
                                    model = row[1]["model"]
                                    api = row[1]["api"]
                                    prediction_set[f"{api}_prediction"] = model.predict(prediction_set[factors])
                                except Exception as e:
                                    print(str(e))
                            prediction_set["prediction"] = (prediction_set["skl_prediction"] + prediction_set["xgb_prediction"] + prediction_set["cat_prediction"]) / 3
                            prediction_set["new_quarter"] = [(row[1]["quarter"] + projection_value) % 4 for row in prediction_set.iterrows()]
                            prediction_set["new_quarter"] = prediction_set["new_quarter"].replace(0,4)
                            prediction_set["new_year"] = [row[1]["year"] if row[1]["quarter"] + projection_value <=4 \
                                                          else row[1]["year"]+ 1 for row in prediction_set.iterrows()]
                            prediction_set = prediction_set[["new_year","new_quarter","ticker","prediction"]].rename(columns={"new_year":"year","new_quarter":"quarter"})
                            prediction_set["training_year"] = training_year
                            prediction_set["projection_value"] = projection_value
                            prediction_set = prediction_set.groupby(["year","quarter"]).mean().reset_index()
                            prediction_set["ticker"] = ticker                        
                            strat.store("eps_sim",prediction_set)
                        except Exception as e:
                            print(str(e))
                            continue
                strat.disconnect()
            except Exception as e:
                print(str(e))
                continue
market.disconnect()

  0%|                                                                                                                                                       | 0/1 [00:00<?, ?it/s]

'date'
Found array with 0 sample(s) (shape=(0, 29)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 29)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 29)) while a minimum of 1 is required.
Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=0.
Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=0.
'str' object has no attribute 'predict'
'str' object has no attribute 'predict'
'str' object has no attribute 'predict'
'skl_prediction'
Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=3.
Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=3.
'str' object has no attribute 'predict'
'str' object has no attribute 'predict'
'xgb_prediction'



  0%|                                                                                                                                                       | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction_set